## `Pipelines`

***What is Pipelines?***

- ***Pipelines*** chains together multiple steps so that the output of each step is used as input of the next step.
- ***Pipelines*** makes it easy to apply the same preprocessing to train and test datasets. As example when a application is live on server and it gets some user inputs, then again those inputs needed to be preprocessed like the train test dataset to give predictions/results. So without using ***Pipelines*** the steps needed to be repeat each time.

### Here we will do the task without `Pipelines`

In [1]:
# importing the libraries

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# importing dataset

df = pd.read_csv('datasets/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Dropping the unnecessary columns

df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1, inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [5]:
# Checking for null values

df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

#### Doing train test split

In [6]:
# Creating independent and dependent variables

X = df.drop(columns=['Survived'], axis=1)
y = df['Survived']

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((712, 7), (179, 7))

In [8]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [9]:
# Most frequent value in 'Embarked' column

df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [10]:
# Applying imputation to fill the missing values in 'Age' and 'Embarked' columns
# For 'Age' we use 'mean' to fill the missing values
# For 'Embarked' we will use the 'most_frequent' value appeared in the column i.e. the 'S' value

si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

# train data
X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

# test data
X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [11]:
# one hot encoding on 'Sex' and 'Embarked' column as they are of categorical types.
# "(handle_unknown='ignore')" means if there is an absolute new value occurs in the test dataset 
# then that will be ignored.

ohe_sex = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False, handle_unknown='ignore')

# train data
X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

# test data
X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

C:\Users\Arun\anaconda3\envs\dsc\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Arun\anaconda3\envs\dsc\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
# Now we need to remove the 'Sex','Age','Embarked' columns from the original train test set 
# And create new arrays of the remaining columns

X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])
X_test_rem = X_test.drop(columns=['Sex','Age','Embarked'])

In [13]:
# Now doing concatenation

X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [14]:
X_train_transformed.shape

(712, 10)

### Now making model

In [15]:
clf = DecisionTreeClassifier()
clf

DecisionTreeClassifier()

In [16]:
# training the model

clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [17]:
# Now doing predictions

y_pred = clf.predict(X_test_transformed)

In [18]:
# Checking accuracy of the model

accuracy = accuracy_score(y_test, y_pred)
print(f"The accuracy of the model is: {(accuracy)*100:.2f}%")

The accuracy of the model is: 78.21%


In [19]:
# Now saving this model

try:
    pickle.dump(ohe_sex, open('models/ohe_sex.pkl', 'wb'))
    pickle.dump(ohe_embarked, open('models/ohe_embarked.pkl', 'wb'))
    pickle.dump(clf, open('models/titanic_clf.pkl', 'wb'))


except Exception as err:
    print("Error is: ", err)
else:
    print("Models saved succesfully.")

Models saved succesfully.


**Notes:**

- Here we needed to save the `ohe_sex` and `ohe_embarked` along with the model, so when we will get a new data there the data will be in it's orginal format and we will be needed to do the transformation of that input data to run it in the model.
- Here the `SimpleImputer` we did not saved as it only required when there is a missing value, now as here we will fill the values so we will not leave any column empty, so we did not save that transformer.

In [20]:
df.iloc[330:333]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
330,1,3,female,NaN,2,0,23.2500,Q
331,0,1,male,45.5,0,0,28.5000,S
332,0,1,male,38.0,0,1,153.4625,S
